In [0]:
#math function

def func_xy(X, Y, L):
  X, Y = np.meshgrid(X, Y, L)
  Z = func_tf_xy(X,Y, L)
  return Z,X,Y

def func_tf_xy(x, y, l):
  return func_f(x, y)+l*func_g(x, y)

def func_f(x, y):
  return x*y

def func_g(x, y):
  return ((x+2)*(y+3.5)-200)


def func(x, y, z):
  #return x**2+y**2+z**2
  return func_tf_xy(x, y, z)
  #return ((200/(y+3.5))-2)*y

In [152]:
import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    res = 10000000000000- func(x, y, z)
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 1
    child_num = 3

    space = OrderedDict((
        ('x', (5., 11.)),
        ('y', (14., 16.)),
        ('z', (2.7, 3.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval 6.740822560101948, 14.561844480888137, 2.9381527373930045: 10000000000025.61
Eval 8.270697268977337, 15.9777066422358, 2.894110269667816: 9999999999867.709
Eval 6.721622690044503, 15.457254078700986, 2.9971263940865818: 9999999999999.988
Best [8.270697268977337, 15.9777066422358, 2.894110269667816]

